# Importando bibliotecas necessárias

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install PyMuPDF
!pip install plac

# Bibliotecas necessárias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import spacy
from spacy.lang.pt import stop_words
import random
import json
import os
import re
from itertools import chain
import sys
from __future__ import unicode_literals, print_function
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import fitz
import plac
import numpy as np
import pickle
import base64
import csv

csv.field_size_limit(sys.maxsize)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 54.2 MB/s eta 0:00:00


In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
!pip install unidecode
from unidecode import unidecode

In [ ]:
!pip install gensim
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

In [ ]:
# Garbage Collector - use it like gc.collect()
import gc

# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [ ]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Partindo frases

In [ ]:
# Ajusta a formatação dos textos
def remove_formatting(text):
    text = text.replace("\n", " ").replace("\t", " ")
    while text.find("  ") != -1:
        text = text.replace("  ", " ")
    return text

In [ ]:
NLP = spacy.load("pt_core_news_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
NLP.enable_pipe("senter")

def get_sentences(text: str):
    sentences = [sent.text.strip() for sent in NLP(text).sents]
    sentences = list(filter(lambda s: len(s) > 0, sentences))
    return sentences

# Stemming

In [ ]:
# Remove acentos e passa para lowercase
formatar = lambda palavra: unidecode(str(palavra).lower())

STEMMER = SnowballStemmer("portuguese", ignore_stopwords=True)
STOPWORDS = stopwords.words('portuguese')

def extract_stem_tokens(sentence: str):
    tokens = [formatar(t) for t in word_tokenize(sentence)]
    important_tokens = list(filter(lambda t: not t in STOPWORDS and t.isalpha(), tokens))
    stems = [STEMMER.stem(t) for t in important_tokens]
    return stems

# Word2Vec + TF-IDF

In [ ]:
# Word2Vec
print("Carregando modelo Word2Vec...")
WORD2VEC = Word2Vec.load("/content/drive/Shareddrives/IA 2023 - Projeto 1 Grupo 3/Modelos/word2vec.model")

# TF-IDF
tf_idf_weights = {}
with open("/content/drive/Shareddrives/IA 2023 - Projeto 1 Grupo 3/Modelos/tf_idf_weights.csv", 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in tqdm(csv_reader, desc="Carregando tokens TF-IDF", position=0, leave=True):
        tf_idf_weights[row[0]] = float(row[1])

In [ ]:
def vectorize_sentence(sentence: str):
    return vectorize_tokens(extract_stem_tokens(sentence))

def vectorize_tokens(tokens: list):
    if not tokens:
        return np.zeros_like(WORD2VEC.wv[0])
    tokens = list(filter(lambda t: (t in WORD2VEC.wv) and (t in tf_idf_weights.keys()), tokens))
    #tfidf = np.sqrt([[tf_idf_weights[t]] for t in tokens])
    tfidf = np.array([[tf_idf_weights[t]] for t in tokens])
    tfidf = np.log(tfidf / np.min(tfidf)) + 1
    w2v = np.array([WORD2VEC.wv[t] for t in tokens])
    return np.sum((w2v * tfidf) / sum(tfidf), axis=0) if len(tokens) > 0 else np.zeros_like(WORD2VEC.wv[0])

# Base de dados

In [ ]:
with open("/content/drive/Shareddrives/IA 2023 - Projeto 1 Grupo 3/Dados/dataset_full.csv", 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    sentence_data = [row for row in tqdm(csv_reader, desc="Carregando base de dados", position=0, leave=True)]

Carregando base de dados: 498219it [00:06, 76227.92it/s]


In [ ]:
lista_modalidades = [
    'Pregão Presencial - Ata de Registro de Preços', 'Ata de Registro de Preço', 'Seleção Pública Simplificada', 'Aviso de Chamamento Público',
    'Dispensa de Licitação', 'Tomada de Preço', 'Ata de Registro de Preços', 'Chamada Pública', 'Pregão', 'Convite', 'Concorrência',
    'Credenciamento', 'Inexigibilidade', 'Pregão Presencial', 'Regime Diferenciado de Contratação', 'Pregão Eletrônico'
]
lista_tipo_publicacao = [
    'Convocação de Licitante', 'Resultado de Julgamento', 'Termo de Comodato', 'Emenda Parlamentar', 'Nomeação Conselho', 'Anulação de ato',
    'Publicação de resultado', 'Processo Administrativo', 'Extrato de ata de registro de preços', 'Requerimento', 'Instrução normativa',
    'Lei Aldir Blanc', 'Proposta de Lei', 'Sessão Solene', 'Extrato de Termos de Compromisso', 'Cooperação Técnica',
    'Autorização de movimentação bancária', 'Ata da Sessão Pública', 'Regimento', 'Convênio', 'Memorial Descritivo', 'Anexos', 'Resolução',
    'Aviso de Demolição - Suspensão', 'Aprovação da Programação Anual de Saúde', 'Cessão de Uso', 'Autorização de uso de imóvel', 'Notificação',
    'Extrato de ata', 'Leilão', 'Manifestação de interesse social', 'Licenciamento Ambiental', 'Rescisão Contratual', 'Programação financeira',
    'Homologação', 'Portaria', 'Convocação para participar de Conselho Municipal', 'Inexigibilidade', 'Aviso de Demolição', 'Termo Aditivo',
    'Ata de abertura de envelope', 'Processo de impeachment', 'Aviso de Licitação', 'Concessão de Patrocínio', 'Processo Administrativo Sanitário',
    'Nota Informativa', 'Processo Seletivo', 'Licitação Deserta', 'Errata', 'Extrato de Contrato', 'Lançamento de tributos', 'Edital de Proclamas',
    'Termo de ajuste de contas', 'Promoção de religião', 'Atestado publicação RGF', 'Desapropriação', 'Revogação', 'Decreto', 'Termo de Fomento',
    'Atos de Pessoal', 'Autorização de Aplicação e Resgate', 'Credenciamento', 'Sanção contratual', 'Carta de Advertência',
    'Reequilíbrio Econômico Financeiro', 'Termo de Colaboração', 'Certidão', 'Execução de Pagamento', 'Cancelamento ou adiantamento indeterminado',
    'Conferência Municipal', 'Parcelamento e Confissão de Débitos Previdenciários', 'Aviso de Advertência', 'Adesão a ata de registro de preços',
    'Concurso Público', 'Parecer em credenciamento', 'Ratificação de dispensa', 'Campeonato esportivo', 'Concurso cultural', 'Audiência Pública',
    'Eleições de membros', 'Julgamento de Contas', 'Ata de Reunião', 'Contencioso Administrativo Fiscal', 'Lei', 'Precatórios FUNDEF',
    'Promulgação de Lei', 'Protocolo de Intenções', 'Cronograma de datas para tramitação de Lei', 'Certidão de Regularização Fundiária',
    'Prorrogação de credenciamento', 'Notificação de recebimento de recursos'
]

In [ ]:
dataset_tem_modalidades = {"x": [], "y": []}
dataset_tem_publicacoes = {"x": [], "y": []}
dataset_tem_festividade = {"x": [], "y": []}
dataset_modalidades = {"x": [], "y": []}
dataset_publicacoes = {"x": [], "y": []}

for x in tqdm(sentence_data):
    try:
        inp = vectorize_tokens(x[1].split())
    except:
        continue

    if inp is None or np.isnan(np.min(inp)):
        continue

    out_modalidades = np.array([1 if x[2] == m else 0 for m in lista_modalidades], dtype=np.float32)
    out_publicacoes = np.array([1 if x[3] == p else 0 for p in lista_tipo_publicacao], dtype=np.float32)

    if np.max(out_modalidades) > 0:
        dataset_modalidades["x"] += [inp]
        dataset_modalidades["y"] += [out_modalidades]

    if np.max(out_publicacoes) > 0:
        dataset_publicacoes["x"] += [inp]
        dataset_publicacoes["y"] += [out_publicacoes]

    dataset_tem_modalidades["x"] += [inp]
    dataset_tem_modalidades["y"] += [np.array([np.max(out_modalidades)])]

    dataset_tem_publicacoes["x"] += [inp]
    dataset_tem_publicacoes["y"] += [np.array([np.max(out_publicacoes)])]

    dataset_tem_festividade["x"] += [inp]
    dataset_tem_festividade["y"] += [np.array([1 if x[4] == "True" else 0])]

100%|██████████| 498219/498219 [01:55<00:00, 4317.77it/s]


In [ ]:
def dividir_treino_teste(dataset, divisao=0.7):
    dataset_x = dataset["x"]
    dataset_y = dataset["y"]
    amostras_treino = int(len(dataset_x) * divisao)

    return {
        "treino_x": np.array(dataset_x[:amostras_treino]),
        "treino_y": np.array(dataset_y[:amostras_treino]),
        "teste_x": np.array(dataset_x[amostras_treino:]),
        "teste_y": np.array(dataset_y[amostras_treino:]),
        "input": dataset_x[0].shape,
        "output": dataset_y[0].shape[0]
    }

In [ ]:
dataset_modalidades = dividir_treino_teste(dataset_modalidades)
dataset_publicacoes = dividir_treino_teste(dataset_publicacoes)
dataset_tem_modalidades = dividir_treino_teste(dataset_tem_modalidades)
dataset_tem_publicacoes = dividir_treino_teste(dataset_tem_publicacoes)
dataset_tem_festividade = dividir_treino_teste(dataset_tem_festividade)

# Treinamento dos Modelos

In [ ]:
# Publicações
dataset = dataset_publicacoes
model = keras.Sequential([
    layers.Input(shape=dataset["input"]),
    layers.Normalization(),
    layers.Dense(64, activation='gelu'),
    layers.Dense(128, activation='gelu'),
    layers.Dense(64, activation='gelu'),
    layers.Dense(dataset["output"], activation='softmax')
])
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
# Modalidades
dataset = dataset_modalidades
model = keras.Sequential([
    layers.Input(shape=dataset["input"]),
    layers.Normalization(),
    layers.Dense(16, activation='gelu'),
    layers.Dense(32, activation='gelu'),
    layers.Dense(16, activation='gelu'),
    layers.Dense(dataset["output"], activation='softmax')
])
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
# Modalidades head
dataset = dataset_tem_publicacoes
model = keras.Sequential([
    layers.Input(shape=dataset["input"]),
    layers.Normalization(),
    layers.Dense(32, activation='gelu'),
    layers.Dense(dataset["output"], activation='sigmoid')
])
model.compile(optimizer=keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
# Festividade head
dataset = dataset_tem_festividade
model = keras.Sequential([
    layers.Input(shape=dataset["input"]),
    layers.Normalization(),
    layers.Dense(32, activation='gelu'),
    layers.Dense(dataset["output"], activation='sigmoid')
])
model.compile(optimizer=keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
# Publicações head
dataset = dataset_tem_publicacoes
model = keras.Sequential([
    layers.Input(shape=dataset["input"]),
    layers.Normalization(),
    layers.Dense(32, activation='gelu'),
    layers.Dense(8, activation='gelu'),
    layers.Dense(4, activation='gelu'),
    layers.Dense(dataset["output"], activation='sigmoid')
])
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [ ]:
for i in range(100):
    history = model.fit(
        dataset["treino_x"], dataset["treino_y"],
        validation_data=(dataset["teste_x"], dataset["teste_y"]),
        epochs=1,
        batch_size=64,
        validation_steps=8
    )
    model.save(f"/content/drive/Shareddrives/IA 2023 - Projeto 1 Grupo 3/Modelos/publicacoes_e{(i+1)}.keras")